# Agno Framework

This notebook covers the Agno framework - an incredibly fast multi-agent framework, runtime and control plane for building AI agents:

- **What is Agno?**: A high-performance multi-agent framework with AgentOS runtime
- **Core Features**: Tools, Knowledge Bases, Memory, MCP Support, and Human-in-the-Loop
- **AgentOS**: FastAPI runtime with integrated control plane
- **Tools**: Extending agent capabilities with custom functions and MCP tools
- **Knowledge**: Integrating knowledge bases for domain-specific information
- **Memory**: Managing conversation history and context with database integration
- **Multi-Agent Systems**: Building agent teams and workflows
- **Practical Applications**: Building real-world agent applications

## Learning Objectives

- Understand what Agno is and why it's useful for building multi-agent systems
- Learn how to create agents with tools, knowledge, and memory using Agno
- Build custom tools to extend agent capabilities
- Integrate knowledge bases and databases for persistent memory
- Use AgentOS for serving agents with FastAPI
- Build multi-agent teams and workflows
- Understand MCP (Model Context Protocol) integration



## Installation

Run this cell to install required packages (uncomment if needed):


In [ ]:
# Install packages (uncomment if needed)
# !pip install agno python-dotenv


## 1. What is Agno?

**Agno** is an incredibly fast multi-agent framework, runtime and control plane. According to the [official documentation](https://docs.agno.com/introduction), it's designed for building multi-agent systems with:

### Key Features:

1. **Multi-Agent Systems**: Build agent teams and workflows
2. **Tools**: Extend agent capabilities with custom functions and MCP (Model Context Protocol) support
3. **Knowledge**: Integrate knowledge bases for domain-specific information
4. **Memory**: Manage conversation history with database integration (SQLite, etc.)
5. **Human-in-the-Loop**: Support for human interaction and feedback
6. **AgentOS**: High-performance FastAPI runtime with integrated control plane
7. **MCP Support**: Model Context Protocol integration for tool connectivity
8. **Private by Design**: Runs entirely in your cloud for complete data privacy

### Why Agno?

- **Speed**: Incredibly fast framework optimized for performance
- **Complete Solution**: Framework + Runtime + Control Plane in one
- **Production-Ready**: Pre-built FastAPI app gets you building AI products on day one
- **Privacy**: AgentOS runs in your cloud, no data leaves your system
- **Control Plane**: Integrated UI for testing, monitoring, and managing systems
- **Flexibility**: Orchestrate agents as teams (autonomy) or step-based workflows (control)

### Agno Architecture:

```
┌─────────────────┐
│   AgentOS       │  ← FastAPI Runtime + Control Plane
└─────────────────┘
       ↓
┌─────────────────┐
│   Agents        │  ← Multi-agent teams or workflows
└─────────────────┘
       ↓
┌─────────────────┐
│   Tools + MCP   │  ← Custom functions and MCP servers
└─────────────────┘
       ↓
┌─────────────────┐
│   Knowledge     │  ← Vector stores, databases, documents
└─────────────────┘
       ↓
┌─────────────────┐
│   Memory (DB)   │  ← Persistent conversation history
└─────────────────┘
```

### When to Use Agno?

- Building multi-agent systems that need high performance
- Need a complete solution (framework + runtime + control plane)
- Require privacy and data control (cloud-based deployment)
- Want to orchestrate agents as teams or workflows
- Building production AI products that need monitoring and management


## 2. Setting Up Agno

First, let's import the Agno framework and set up our environment. Agno provides a clean API for building agents.


In [ ]:
# Import Agno framework
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Agno core imports
from agno.agent import Agent
from agno.db.sqlite import SqliteDb
from agno.models.openai import OpenAI
from agno.models.anthropic import Claude
from agno.os import AgentOS
from agno.tools.mcp import MCPTools

print("✅ Agno framework imported successfully!")
print("\nAvailable components:")
print("- Agent: Core agent class")
print("- SqliteDb: SQLite database for memory")
print("- OpenAI/Claude: LLM models")
print("- AgentOS: FastAPI runtime")
print("- MCPTools: Model Context Protocol tools")


## 3. Creating Your First Agno Agent

Let's create a simple Agno agent with memory, tools, and knowledge. Agno makes it easy to build agents with all these features.


In [ ]:
# Create an Agno agent with memory, tools, and knowledge
# This example shows the basic structure from the Agno documentation

# Check for API keys
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# Create agent with SQLite database for memory
agno_agent = Agent(
    name="Agno Learning Agent",
    # Use OpenAI or Claude - uncomment based on which API key you have
    model=OpenAI(id="gpt-4o-mini") if openai_api_key else None,
    # model=Claude(id="claude-sonnet-4-5") if anthropic_api_key else None,
    
    # Database for persistent memory
    db=SqliteDb(db_file="agno_learning.db"),
    
    # Add conversation history to context automatically
    add_history_to_context=True,
    
    # Enable markdown formatting in responses
    markdown=True,
    
    # Instructions for the agent
    instructions="You are a helpful AI assistant learning about the Agno framework. "
                 "You help users understand how to build agents with tools, knowledge, and memory."
)

print("✅ Agno agent created!")
print(f"Agent name: {agno_agent.name}")
print(f"Database: agno_learning.db (for persistent memory)")
print(f"History to context: {agno_agent.add_history_to_context}")


## 4. Tools: Extending Agent Capabilities

Agno supports custom tools and MCP (Model Context Protocol) tools. Tools allow agents to interact with the external world - APIs, databases, file systems, etc.

### 4.1 Custom Tools

Let's create custom tools for the agent.


In [ ]:
# Agno uses function-based tools with proper type hints and docstrings
# The framework automatically converts Python functions into tools

# Example: Simple calculator function
def calculate(operation: str, a: float, b: float) -> float:
    """Perform a mathematical calculation.
    
    Args:
        operation: The operation (add, subtract, multiply, divide)
        a: First number
        b: Second number
    
    Returns:
        The result of the calculation
    """
    operations = {
        "add": lambda x, y: x + y,
        "subtract": lambda x, y: x - y,
        "multiply": lambda x, y: x * y,
        "divide": lambda x, y: x / y if y != 0 else None
    }
    
    if operation.lower() not in operations:
        raise ValueError(f"Unknown operation: {operation}")
    
    result = operations[operation.lower()](a, b)
    if result is None:
        raise ValueError("Division by zero")
    return result

# Example: Weather tool (simulated)
def get_weather(city: str) -> str:
    """Get weather information for a city.
    
    Args:
        city: Name of the city
    
    Returns:
        Weather information
    """
    weather_data = {
        "new york": "Sunny, 72°F",
        "london": "Cloudy, 15°C",
        "tokyo": "Rainy, 18°C",
        "paris": "Partly cloudy, 20°C"
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        return f"Weather in {city}: {weather_data[city_lower]}"
    else:
        available = ", ".join(weather_data.keys())
        return f"Weather data not available for {city}. Available cities: {available}"

# Add tools to the agent
if agno_agent.model:
    # Agno automatically converts functions to tools when added
    agno_agent.tools = [calculate, get_weather]
    print("✅ Tools added to agent!")
    print("Available tools:")
    print("- calculate(operation, a, b): Perform calculations")
    print("- get_weather(city): Get weather information")
else:
    print("⚠️  Agent model not configured. Set OPENAI_API_KEY or ANTHROPIC_API_KEY")


## 5. MCP Tools: Model Context Protocol

Agno supports MCP (Model Context Protocol) tools, which allow agents to connect to external MCP servers for additional capabilities. This is a powerful feature for extending agent functionality.


In [ ]:
# Example: Adding MCP tools to an agent
# MCP (Model Context Protocol) allows agents to connect to external servers

# Create an agent with MCP tools (example from Agno docs)
mcp_agent = Agent(
    name="MCP Agent",
    model=OpenAI(id="gpt-4o-mini") if openai_api_key else None,
    db=SqliteDb(db_file="mcp_agent.db"),
    tools=[
        # MCPTools connects to an MCP server
        # Example: MCPTools(transport="streamable-http", url="https://docs.agno.com/mcp")
        # Uncomment and configure when you have an MCP server available
    ],
    add_history_to_context=True,
    markdown=True,
)

print("✅ MCP agent structure created!")
print("\nNote: To use MCP tools, you need:")
print("1. An MCP server running")
print("2. Configure MCPTools with the server URL")
print("3. See Agno documentation for MCP setup details")


## 6. Memory: Database-Persisted Conversation History

Agno uses databases (like SQLite) to persist conversation history. This allows agents to maintain context across sessions and provides persistent memory.


In [ ]:
# Memory in Agno is handled automatically through the database
# When you set add_history_to_context=True, Agno automatically includes
# conversation history in the context for each new message

print("Agno Memory Features:")
print("1. ✅ Persistent Storage: Conversations stored in SQLite database")
print("2. ✅ Automatic Context: History automatically added to context (if enabled)")
print("3. ✅ Session Management: Maintains context across multiple interactions")
print("4. ✅ Query History: Can query past conversations from database")
print("\nThe agent automatically:")
print("- Stores all messages in the database")
print("- Retrieves relevant history when add_history_to_context=True")
print("- Maintains conversation state across sessions")
print("- Provides persistent memory that survives restarts")

# The database file is created automatically
# You can query it directly or use Agno's built-in methods
print(f"\nDatabase location: agno_learning.db")
print("This file persists all conversation history!")


## 7. Using the Agent

Now let's use the agent we created. Agno agents have a simple API for running conversations.


In [ ]:
# Run the agent with a query
# Agno agents use the run() method for synchronous execution

if agno_agent.model:
    print("=" * 60)
    print("Example 1: Basic Agent Query")
    print("=" * 60)
    
    # Run a simple query
    response = agno_agent.run("What is the Agno framework?")
    print(f"\nAgent Response:\n{response.content}\n")
    
    print("✅ Query completed! Response stored in database.")
else:
    print("⚠️  Agent model not configured.")
    print("Please set OPENAI_API_KEY or ANTHROPIC_API_KEY in your .env file")
    print("Example: export OPENAI_API_KEY='your-key-here'")


## 8. Testing Tools and Memory

Let's test the agent's tools and memory capabilities.


In [ ]:
# Example 2: Using Tools
if agno_agent.model:
    print("=" * 60)
    print("Example 2: Using Tools")
    print("=" * 60)
    
    # The agent will automatically use tools when needed
    response = agno_agent.run("What is 25 multiplied by 17? Use the calculate tool.")
    print(f"\nAgent Response:\n{response.content}\n")
else:
    print("⚠️  Agent not initialized.")


In [ ]:
# Example 3: Testing Memory
if agno_agent.model:
    print("=" * 60)
    print("Example 3: Testing Memory")
    print("=" * 60)
    
    # First message - establish context
    response1 = agno_agent.run("My name is Alice and I'm a data scientist.")
    print(f"\nFirst Response:\n{response1.content}\n")
    
    # Second message - should remember the first
    response2 = agno_agent.run("What's my profession?")
    print(f"\nSecond Response (using memory):\n{response2.content}\n")
    
    print("✅ Memory test completed! The agent remembered the previous conversation.")
else:
    print("⚠️  Agent not initialized.")


In [ ]:
# Example 4: Combining Tools and Memory
if agno_agent.model:
    print("=" * 60)
    print("Example 4: Combining Tools and Memory")
    print("=" * 60)
    
    response = agno_agent.run("What's the weather in New York?")
    print(f"\nAgent Response:\n{response.content}\n")
else:
    print("⚠️  Agent not initialized.")


In [ ]:
## 9. AgentOS: FastAPI Runtime

AgentOS provides a FastAPI runtime for serving agents. This allows you to build production-ready AI applications.


## 9. AgentOS: FastAPI Runtime

AgentOS provides a FastAPI runtime for serving agents. This allows you to build production-ready AI applications with a web interface.


In [ ]:
# Create AgentOS instance with our agent
# AgentOS provides a FastAPI app that you can serve

agent_os = AgentOS(agents=[agno_agent])

# Get the FastAPI app
app = agent_os.get_app()

print("✅ AgentOS created!")
print("\nAgentOS Features:")
print("1. FastAPI runtime for serving agents")
print("2. REST API endpoints for agent interactions")
print("3. WebSocket support for real-time communication")
print("4. Integrated control plane (AgentOS UI)")
print("\nTo serve the agent:")
print("  agent_os.serve(app='agno_framework:app', reload=True)")
print("\nOr use the AgentOS UI at: https://os.agno.com")
print("The UI connects directly to your runtime for testing and monitoring.")


## 10. Multi-Agent Systems

Agno supports building multi-agent teams and workflows. You can orchestrate agents as autonomous teams or step-based workflows.


In [ ]:
# Example: Creating a multi-agent team
# Each agent can have different roles and capabilities

# Research Agent
research_agent = Agent(
    name="Research Agent",
    model=OpenAI(id="gpt-4o-mini") if openai_api_key else None,
    db=SqliteDb(db_file="research_agent.db"),
    instructions="You are a research assistant. You help find and analyze information.",
    add_history_to_context=True,
)

# Analysis Agent
analysis_agent = Agent(
    name="Analysis Agent",
    model=OpenAI(id="gpt-4o-mini") if openai_api_key else None,
    db=SqliteDb(db_file="analysis_agent.db"),
    instructions="You are an analyst. You analyze data and provide insights.",
    add_history_to_context=True,
)

# Create AgentOS with multiple agents
multi_agent_os = AgentOS(agents=[research_agent, analysis_agent])

print("✅ Multi-agent system created!")
print("\nAgents in the system:")
print(f"1. {research_agent.name} - Research and information gathering")
print(f"2. {analysis_agent.name} - Data analysis and insights")
print("\nYou can orchestrate these agents as:")
print("- Autonomous teams (agents work independently)")
print("- Step-based workflows (agents work in sequence)")


In [ ]:
## 11. Best Practices

Here are best practices for building agents with Agno based on the [official documentation](https://docs.agno.com/introduction):


## 11. Best Practices

Here are best practices for building agents with Agno based on the [official documentation](https://docs.agno.com/introduction):

### Agent Design
- **Clear Instructions**: Write clear, specific instructions for agent behavior
- **Database for Memory**: Always use a database (SqliteDb) for persistent memory
- **History Context**: Enable `add_history_to_context=True` for conversational agents
- **Markdown Support**: Enable `markdown=True` for better formatted responses

### Tools
- **Function-Based**: Use Python functions with proper type hints and docstrings
- **MCP Integration**: Leverage MCP tools for connecting to external services
- **Error Handling**: Always handle errors gracefully in tool functions
- **Documentation**: Document tool parameters and return values clearly

### Memory & Database
- **Persistent Storage**: Use SqliteDb or other database backends for persistence
- **Context Management**: Let Agno handle context automatically with `add_history_to_context`
- **Session Management**: Database automatically manages conversation sessions
- **Privacy**: Data stays in your cloud - no external services needed

### AgentOS & Deployment
- **FastAPI Runtime**: Use AgentOS for production deployments
- **Control Plane**: Leverage AgentOS UI for testing and monitoring
- **Multi-Agent**: Design agent teams for complex workflows
- **Privacy First**: AgentOS runs in your cloud for complete data control

### Performance
- **Fast Framework**: Agno is optimized for speed - leverage its performance
- **Efficient Queries**: Database queries are optimized automatically
- **Scalable**: AgentOS can handle multiple agents and concurrent requests


## 12. Complete Example: Production-Ready Agent

Here's a complete example from the Agno documentation showing how to build a production-ready agent with MCP, database, and AgentOS.


In [ ]:
# Complete example from Agno documentation
# This shows a production-ready setup with MCP, database, and AgentOS

production_agent = Agent(
    name="Production Agent",
    model=OpenAI(id="gpt-4o-mini") if openai_api_key else None,
    # Or use Claude: model=Claude(id="claude-sonnet-4-5") if anthropic_api_key else None,
    
    # Database for persistent memory
    db=SqliteDb(db_file="production_agent.db"),
    
    # MCP tools (uncomment when you have an MCP server)
    # tools=[MCPTools(transport="streamable-http", url="https://docs.agno.com/mcp")],
    
    # Add conversation history to context
    add_history_to_context=True,
    
    # Enable markdown formatting
    markdown=True,
    
    # Custom instructions
    instructions="You are a helpful AI assistant with access to tools and persistent memory."
)

# Create AgentOS for serving
production_os = AgentOS(agents=[production_agent])
production_app = production_os.get_app()

print("✅ Production-ready agent created!")
print("\nTo serve this agent:")
print("  agent_os.serve(app='agno_framework:production_app', reload=True)")
print("\nOr use AgentOS UI:")
print("  https://os.agno.com")
print("\nThis setup provides:")
print("- Persistent memory via SQLite database")
print("- FastAPI runtime for production")
print("- Control plane for monitoring")
print("- MCP support for external tools")
print("- Complete privacy (runs in your cloud)")


In [ ]:
## 13. Summary

In this notebook, we've learned how to build agents using the Agno framework:


## 13. Summary

In this notebook, we've learned how to build agents using the **Agno framework** - an incredibly fast multi-agent framework, runtime and control plane:

### Key Takeaways:

1. **Agno Framework**: High-performance framework for multi-agent systems
   - Fast and optimized for production
   - Complete solution: framework + runtime + control plane
   - Private by design - runs in your cloud

2. **Agents**: Build agents with tools, knowledge, and memory
   - Simple API: `Agent(name, model, db, tools, ...)`
   - Database integration for persistent memory
   - Automatic context management

3. **Tools**: Extend agent capabilities
   - Function-based tools with type hints
   - MCP (Model Context Protocol) support
   - Easy integration with external services

4. **Memory**: Persistent conversation history
   - SQLite database integration
   - Automatic context inclusion
   - Session management

5. **AgentOS**: Production-ready runtime
   - FastAPI application
   - Integrated control plane
   - WebSocket and REST API support

### Next Steps:

- Follow the [Agno quickstart](https://docs.agno.com/introduction/quickstart) for hands-on learning
- Explore the [examples gallery](https://docs.agno.com/examples/introduction) for real-world applications
- Build your own multi-agent systems
- Deploy agents using AgentOS
- Integrate MCP servers for extended capabilities

### Resources:

- [Agno Documentation](https://docs.agno.com/introduction)
- [AgentOS UI](https://os.agno.com) - Control plane for testing and monitoring
- [MCP Documentation](https://modelcontextprotocol.io) - Model Context Protocol
- [Agno Examples](https://docs.agno.com/examples/introduction) - Real-world examples


## Exercises

Try these exercises to practice building agents with Agno:

1. **Custom Tools**: Create custom tools for your specific use case (e.g., API integrations, data processing)
2. **MCP Integration**: Set up an MCP server and connect it to your agent
3. **Multi-Agent Team**: Build a team of specialized agents that work together
4. **AgentOS Deployment**: Deploy your agent using AgentOS and test it via the UI
5. **Database Management**: Explore the SQLite database to understand how memory is stored
6. **Workflow Design**: Create step-based workflows with multiple agents
7. **Production Setup**: Configure a production-ready agent with monitoring and logging

### Getting Started:

1. Install Agno: `pip install agno`
2. Follow the [quickstart guide](https://docs.agno.com/introduction/quickstart)
3. Explore the [examples gallery](https://docs.agno.com/examples/introduction)
4. Build your first agent!

Happy building with Agno! 🚀
